<a href="https://colab.research.google.com/github/torimonn/hands-on/blob/main/PDF%E6%8A%BD%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-google-genai
!pip install -qU PyPDF2
!pip install -qU Python-docx
!pip install -qU docling
!pip install -qU scipy==1.13.3
!pip install -qU pdfplumber
!pip install -qU pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8

In [ ]:
# 必要なライブラリのインポート
import os
import io
import re
import numpy as np
from typing import List
from PyPDF2 import PdfReader
from google.colab import files
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from docx import Document

# Google Colab上でファイルをアップロード
uploaded = files.upload()

# APIキーの設定
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

Saving 6.pdf to 6.pdf


In [ ]:
# pdfplumber
import pdfplumber

def extract_text_and_tables(pdf_path):
    """PDFのテキストと表を順番通りに取得"""
    results = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            results.append(f"--- Page {i+1} ---")  # ページ番号を追加

            # テキストの取得
            text = page.extract_text()
            if text:
                results.append(text)

            # 表の取得
            tables = page.extract_tables()
            for j, table in enumerate(tables):
                results.append(f"[Table {j+1} on Page {i+1}]")
                for row in table:
                    results.append("\t".join(cell if cell else "" for cell in row))  # Noneを空文字に変換

    return "\n".join(results)

# アップロードされたファイルを処理
for file_name, file_content in uploaded.items():
    print(f"Processing file: {file_name}")

    # PDFの処理
    pdf_content = extract_text_and_tables(file_name)

    # .docx ファイルに書き込み
    doc = Document()
    for line in pdf_content.split("\n"):
        doc.add_paragraph(line)  # 改行を保持しながら追加
    doc.save("pdfplumber.docx")

    # .docx ファイルをダウンロード
    files.download("pdfplumber.docx")

Processing file: 6.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# docling
from docling.document_converter import DocumentConverter

# DocumentConverterのインスタンスを作成
converter = DocumentConverter()

# アップロードされたファイルを処理
for file_name, file_content in uploaded.items():
    print(f"Processing file: {file_name}")

    # ドキュメントファイルを変換
    text = converter.convert(file_name)

    # Markdownに変換
    markdown_output = text.document.export_to_markdown()

    # テキストファイルに書き込み
    doc = Document()
    doc.add_paragraph(markdown_output)  # Markdownの内容を Word に追加
    doc.save("docling.docx")

    # .docx ファイルをダウンロード
    files.download("docling.docx")

Processing file: 6.pdf


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# pymupdf4llm
import pymupdf4llm

# アップロードされたファイルを処理
for file_name, file_content in uploaded.items():
    print(f"Processing file: {file_name}")

    # PDFからMarkdownを抽出
    md_text = pymupdf4llm.to_markdown(file_name)

    # テキストファイルに書き込み
    doc = Document()
    doc.add_paragraph(md_text)  # Markdownの内容を Word に追加
    doc.save("pymupdf4llm.docx")

    # .docx ファイルをダウンロード
    files.download("pymupdf4llm.docx")

Processing file: 6.pdf
Processing 6.pdf...
[                                        ] (0/127)[                                        ] (  1/127)[                                        ] (  2/127)[                                        ] (  3/127)=[=                                       ] (  4/127)[=                                       ] (  5/127)[=                                    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# gemini-2.0-flash-thinking-exp-01-21 ※出力トークンは最大6万トークン程度なので、それを超える出力はできません
from google import genai
from google.genai import types

# アップロードされたファイルを処理
for file_name, file_content in uploaded.items():
    print(f"Processing file: {file_name}")
    # Gemini APIクライアントを初期化  ※APIキーを取得して登録しておく必要があります
    client = genai.Client(api_key=GOOGLE_API_KEY)
    # 使用するモデルID
    MODEL_ID = 'gemini-2.0-flash-thinking-exp-01-21'

    # File APIを使用してPDFをアップロード
    file_upload = client.files.upload(path=file_name)

    # Gemini APIにアップロードしたファイルのURIを指定してテキスト化を依頼
    response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_upload.uri,
                    mime_type='application/pdf'
                    ),
                ]
            ),
        types.Content(
            role="user",
            parts=[
                    types.Part(text="""
                    RAGに使うためにPDFファイルから正確な情報をマークダウン形式で抜き出したいです。
                    表は表形式で抜き出してください。
                    画像は画像を説明するテキストで表現してください。(資料の中の空白部分は表現不要です)
                    位置関係を考慮して出力してください。
                     """)
                    ]
            )
        ]
    )

    # APIのレスポンスを処理 (思考部分はスキップ)
    extracted_text = response.candidates[0].content.parts[0].text
    doc = Document()
    doc.add_paragraph(extracted_text)  # Markdownの内容を Word に追加
    doc.save("gemini-2.0-flash.docx")

    # .docx ファイルをダウンロード
    files.download("gemini-2.0-flash.docx")

Processing file: 6.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>